<a href="https://colab.research.google.com/github/aludaiyaarasu/data_anaysis_stock/blob/master/Avwap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
pip install yfinance pandas numpy openpyxl pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=9a3f8ee86d1a0fc4bb5903e9d4a7d6b809e0ac424a2efd32d00de6083767c4f1
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [19]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

# Load stock symbols and market cap data from an Excel file
try:
    stocks_df = pd.read_excel(r'/content/MCAP28032024.xlsx')
except FileNotFoundError:
    print("The file was not found.")
    exit()

# Ensure the DataFrame has the correct column names
stocks_df.columns = ['Symbol', 'MCAP_Cr']
# Filter stocks with market capitalization greater than 500 crores (500 Cr INR)
filtered_stocks = stocks_df[stocks_df['MCAP_Cr'] > 500]

def calculate_vwap(data):
    q = data['Volume']
    p = data['High']
    return (p * q).sum() / q.sum()

def calculate_ema(data, window):
    return data.ewm(span=window, adjust=False).mean()

results = []

for index, row in filtered_stocks.iterrows():
    stock_code = row['Symbol']

    # Fetch the entire historical stock data to get the IPO date
    stock_data = yf.download(stock_code + '.NS', period='max')

    if stock_data.empty:
        continue

    # Get the farthest date available in the historical data as the IPO date
    ipo_date = stock_data.index[0]

    # Use only the last 1 years of data for analysis
    start_date = datetime.now() - timedelta(days=1*365)  # 1 years back
    recent_stock_data = stock_data.loc[start_date:]

    if recent_stock_data.empty:
        continue

    # Get the highest price date and price from the last 1 years
    max_price_date = stock_data['High'].idxmax()  # Index of the row with the highest price
    max_price = stock_data.loc[max_price_date, 'High']  # Highest price on that day

    # Get the highest price on the highest price date
    highest_price_on_max_date = stock_data.loc[max_price_date, 'High']

    # Calculate the anchored VWAP from the highest price date to the current date
    anchored_stock_data = stock_data.loc[max_price_date:]
    anchored_vwap = calculate_vwap(anchored_stock_data)

    # Fetch current stock data
    current_data = stock_data.iloc[-1]  # Last row of the DataFrame for the current data
    current_price = current_data['Close']
    opening_price = stock_data.iloc[-1]['Open']
    closing_price = stock_data.iloc[-1]['Close']
    last_5_days_volume_avg = stock_data['Volume'].tail(5).mean()

    # Check if the current price is within ±10% of the anchored VWAP price
    if 0.9 * anchored_vwap <= current_price <= 1.1 * anchored_vwap:
        # Calculate 10-day, 21-day, and 200-day EMAs
        stock_data['EMA_10'] = calculate_ema(stock_data['Close'], window=10)
        stock_data['EMA_21'] = calculate_ema(stock_data['Close'], window=21)
        stock_data['EMA_200'] = calculate_ema(stock_data['Close'], window=200)

        # Calculate 20-day and 50-day average volume
        stock_data['20-Day Avg Volume'] = stock_data['Volume'].rolling(window=20).mean()
        stock_data['50-Day Avg Volume'] = stock_data['Volume'].rolling(window=50).mean()
        avg_volume_20 = stock_data['20-Day Avg Volume'].iloc[-1]
        avg_volume_50 = stock_data['50-Day Avg Volume'].iloc[-1]
        last_5_days_avg_volume_ratio = last_5_days_volume_avg / avg_volume_50

        # Calculate EMA_10 > EMA_21
        ema_10 = stock_data['EMA_10'].iloc[-1]
        ema_21 = stock_data['EMA_21'].iloc[-1]
        ema_200 = stock_data['EMA_200'].iloc[-1]
        ema_10_greater_than_ema_21 = ema_10 > ema_21
        current_price_greater_than_ema200 = current_price > ema_200
        avg_volume_multiple = current_data['Volume'] / avg_volume_50 if avg_volume_50 != 0 else None

        # Check filter conditions
        filter_condition = all([
            current_price > opening_price,
            avg_volume_multiple> 1,
            ema_10 > ema_21,
            current_price > ema_200
        ])
        # Get the current timestamp
        download_timestamp = datetime.now()

        # Append results to the list
        results.append({
            'Symbol': stock_code,
            'MCAP_Cr': row['MCAP_Cr'],
            'IPO Date': ipo_date,
            'Current Price': current_price,
            'Opening Price': opening_price,
            'Closing Price': closing_price,
            'Anchored VWAP': anchored_vwap,
            'Volume': current_data['Volume'],
            'Highest Price': max_price,
            'Highest Price Date': max_price_date,
            'Highest Price on Max Date': highest_price_on_max_date,
            'EMA_10': ema_10,
            'EMA_21': ema_21,
            'EMA_200': ema_200,
            '20-Day Avg Volume': avg_volume_20,
            '50-Day Avg Volume': avg_volume_50,
            'EMA_10 > EMA_21': ema_10_greater_than_ema_21,
            'Closing Price/Anchored VWAP in %': (current_price / anchored_vwap) * 100,
            'Volume/50-Day Avg Volume': avg_volume_multiple,
            'Current Price > Opening Price': current_price > opening_price,
            'Current Price/Opening Price': current_price / opening_price,
            'Current Price/Anchored VWAP': current_price / anchored_vwap,
            'Last 5 Days Avg Volume': last_5_days_volume_avg,
            'Last 5 Days Avg Volume/50-Day Avg Volume': last_5_days_avg_volume_ratio,
            'Current Price > EMA_200':current_price_greater_than_ema200 ,
            'Filter': filter_condition,
            'Time of Download': datetime.now()

        })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Save the results to a new Excel file
output_path = r'/content/filtered_stocks_results.xlsx'

# Attempt to save the file, try another name if there is a PermissionError
try:
    results_df.to_excel(output_path, index=False)
    print(f"Results saved to {output_path}")
except PermissionError:
    alternative_path = r'/content/filtered_stocks_results_alternative.xlsx'
    results_df.to_excel(alternative_path, index=False)
    print(f"Permission denied for {output_path}. Results saved to {alternative_path}")

# Display the results
print(results_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Results saved to /content/filtered_stocks_results.xlsx
          Symbol       MCAP_Cr   IPO Date  Current Price  Opening Price  \
0       RELIANCE  2.010560e+06 1996-01-01    2960.399902    2952.800049   
1            TCS  1.402479e+06 2002-08-12    4517.700195    4510.250000   
2       HDFCBANK  1.099915e+06 1996-01-01    1663.599976    1674.000000   
3      ICICIBANK  7.676568e+05 2002-07-01    1248.449951    1252.000000   
4     BHARTIARTL  6.947840e+05 2002-07-01    1634.849976    1651.000000   
...          ...           ...        ...            ...            ...   
1047  PSUBANKADD  1.500000e+03 2023-08-04      67.129997      67.669998   
1048      MAFANG  1.500000e+03 2021-05-14      94.449997      94.489998   
1049    MOREALTY  1.500000e+03 2024-03-21     105.360001     103.779999   
1050  SHARIABEES  1.500000e+03 2009-03-18     601.859985     599.650024   
1051      MON100  1.500000e+03 2011-03-29     158.779999     159.000000   

      Closing Price  Anchored VWAP     Volum